## Bedrock model integration with Langchain Agents

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

In this notebook, we will demonstrate the use `Zero-shot ReAct` and uses the [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) framework to select the appropriate tool based exclusively on the tool's description. It requires you provide the description of each tool. 

## Prerequisites
* **Install Dependencies**: Be sure to run the [setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) and install the dependencies.
* **IAM Permissions**: The IAM profile running this notebook will need ReadOnly access to EC2. The best way to do this is attach the AmazonEC2ReadOnlyAccess policy to the IAM role this notebook is running under.
* **Create an EC2 instance** with a tag name of 'delete' and a value of 'this instance should be stopped'. Once the instance has been started, set the state to stopped.

## Setup

In [ ]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain


In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)


## Custom Tools

You can introduce your own tools within the agent to perform specific actions. These could consist of looking up data in a database, or making API calls. In this example, we'll stub out some tools that would make API calls on behalf of the agent. For simplicity, the code returns a hardcoded value, but experiment to create your own integrations.

The example below pulls in two built-in tools for google search (SerpAPI) and a calculator (LLMMathChain). It also adds 4 custom tools:
**EC2Search**: Simulates a search for EC2 instances given a tag name.
**EC2Patch**: Simulates patching an EC2 instance.
**EC2Stop**: Simulates shutting down an EC2 instance.
**ChangeRecord**: Simulates writing a change record to a CMDB.

Notice how the description provides context to when to use the tool. That is a key component - it uses the LLM to determine which tool, if any, best solves the question being asked. You don't need a word-for-word match, it will do its best to model which of the tools is the best fit.


In [ ]:
import json

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=False)
tools = load_tools(["llm-math"], llm=llm)
tools.append(Tool.from_function(
        name="EC2Search",
        func=lambda x: f"['i-00000000000','i-00000000001','i-00000000002']",  # Mock Function, replace with a boto3 call
        description="Use this when you need to list EC2 instances in json. It takes a single parameter tagname"
    ))
tools.append(Tool.from_function(
        name="EC2Volumes",
        func=lambda x: f"['vol-06a9a75b13a3a754f']",  # Mock Function, replace with a boto3 call
        description="Use this when you need to list the EC2 volumes given the instance id. It takes a single parameter instanceid"
    ))
tools.append(Tool.from_function(
        name="EC2Stop",
        func=lambda x: f"{len(x.split(','))} instances stopped",  # Mock Function, replace with a boto3 call
        description="Use this when you need to stop EC2 instances. It takes a single parameter instanceid."
    ))

def record_change(x):
    print(f"*{x}*")
    j = json.loads(x)
    return f"instance {j['instance']}. ACTION: {j['changeType']} recorded in CMDB" # Mock Function, replace with an API call

tools.append(Tool.from_function(
        name="ChangeRecord",
        func=record_change,  # Mock Function, replace with an api call
        description="Use this when you need to update a change record in CMDB. This takes in a json document as the parameter. The element named 'instance' contains the instance and the element named 'changeType' is the change type."
    ))

In [ ]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, list the volumes of each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## Generating the tool code
Could you use GenAI to author the code in the tools? Sure! With caution, there is some variation in the code and 
may not be safe for execution without a human review.

The following example uses the code generated by Claude to generate the tool. It replaces the EC2Search tool above with the boto python code.

In [ ]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter. 

return the code inside <code></code>.
Computer:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)



## Executing the code
If the code above looks reasonable, we can use it to run our agent.

**Note - for this to work, you need an EC2 instance in this account with a tag named 'delete' and any value in the tag. It is case senstitive, so prior to executing this, create an EC2 instance and set it to the stopped state.**

In [ ]:
for tool in tools:
    if tool.name == "EC2Search":
        tool.func = list_tagged_instances
        
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. Assistant:"""

result = react_agent.run(question)

print(f"{result}")

In [ ]:
def list_volumes(instance_id):
    ec2 = boto3.resource('ec2')
    instance = ec2.Instance(instance_id)
    retval = []
    for volume in instance.volumes.all():
        retval.append({"id": volume.id, "size": volume.size})
    
    return retval

for tool in tools:
    if tool.name == "EC2Volumes":
        tool.func = list_volumes



In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, list the volumes of each of the instances. 
Finally, report the id and size for each of the volumes in markdown grid by passing in an array of json objects with an element of id and size.

Assistant:"""

result = react_agent.run(question)

Markdown(f"{result}")

## Database Tools
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

### SQL Database Agent
Langachain has a SQL Database agent for demonstrating how to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/integrations/toolkits/sql_database

The agent will load the schema of the DB into context and generate SQL statements based on natural language questions. The SQL statement is then executed against the database and the results returned.

### Data Agents
While the SQL Database agent is useful for data exploration and generating queries, there are also cases where you want to 
For specific entities, a tool can be created to pull data from the database to provide context next steps in the prompt.

The following example will simulate a DB query for a customer in the customer table. Replace this code with a lookup ib DynamoDB or a relation database.

You can see in the ```db_module.py``` just returns from an array held in memory, but replacing this code with a boto call to retreive a DynamoDb record would have the same effect.

In [ ]:
from db_module import  customer_lookup

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))


In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: write one sentence summary including order information about the customer with an id of 8.

Assistant:"""

result = react_agent.run(question)

print(f"{result}")